In [19]:
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
%matplotlib inline
import dateutil.parser as parser
import gc

In [20]:
data_path = "/workspace/data"

In [21]:
s2_identifier = 'S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

In [22]:
lon = 68.08586

In [23]:
lat = 37.20143

In [24]:
extent = 0.02

In [25]:
s2_reference = ''

In [26]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL1C.xml" % (data_path, s2_identifier, s2_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

In [30]:
print(product.getStartTime())
print(product.getEndTime())

09-SEP-2017 06:06:31.026000
09-SEP-2017 06:06:31.026000


In [31]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

parameters = HashMap()
parameters.put('referenceBand', 'B4')

resample = snappy.GPF.createProduct('Resample', parameters, product)

In [32]:
savi_expr = '! opaque_clouds_10m and ! cirrus_clouds_10m and ! nodata_B8 and ! defective_B8 ? (1+0.16) * (B8 - B4)/(B8 + B4 + 0.16) : -999'

In [33]:
cloud_expr = 'opaque_clouds_10m ? 1 : 0'

In [34]:
BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand0 = BandDescriptor()
targetBand0.name = 'savi'
targetBand0.type = 'float32'
targetBand0.expression = savi_expr

targetBand1 = BandDescriptor()
targetBand1.name = 'cloud_mask'
targetBand1.type = 'float32'
targetBand1.expression = cloud_expr

targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 2)
targetBands[0] = targetBand0
targetBands[1] = targetBand1

parameters = HashMap()
parameters.put('targetBands', targetBands)

In [35]:
bandmath = snappy.GPF.createProduct('BandMaths', parameters, product)

In [36]:
p1 = str(lon - extent) + ' ' + str(lat - extent)
p2 = str(lon + extent) + ' ' + str(lat - extent)
p3 = str(lon + extent) + ' ' + str(lat + extent)
p4 = str(lon - extent) + ' ' + str(lat + extent)

wkt = 'POLYGON((%s, %s, %s, %s, %s))' % (p1, p2, p3, p4, p1)

In [37]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)

subset = snappy.GPF.createProduct('Subset', parameters, bandmath)

In [17]:
resample = None
bandmath = None
gc.collect()

0

In [18]:
snappy.ProductIO.writeProduct(subset, 'savi.tif', 'GeoTIFF')

In [49]:
parser.parse(subset.getEndTime().toString()).isoformat()

'2017-09-09T06:06:31.026000'

In [52]:
properties = open('savi.tif.properties', 'w')


In [53]:
properties.write('date=%s/%s' % (parser.parse(subset.getStartTime().toString()).isoformat(), 
                                 parser.parse(subset.getEndTime().toString()).isoformat()))

In [54]:
properties.close()